# [2주차]_기본과제_주어진_문장에서_나올_다음_단어를_예측하는_모델_구현

질문
1. collate_fn 함수에서 labels.append(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[-3]) 이걸 왜 하는 건가요? label을 input_ids에서 뒤에서 3번째 토큰을 사용하는 이유를 모르겠습니다.

2. decoder 부분이 없는거 같은데, transformer라고 볼 수 있나요?

3. task가 분류가 아닌데 CrossEntropyLoss를 사용해도 되나요?

In [ ]:
!pip install datasets sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from torch.nn.utils.rnn import pad_sequence


# ds = load_dataset("stanfordnlp/imdb")
train_ds = load_dataset("stanfordnlp/imdb", split="train[:5%]")
test_ds = load_dataset("stanfordnlp/imdb", split="test[:5%]")

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[-3]) # input_ids에서 뒤에서 3번째 토큰을 사용. why..?
    texts.append(torch.LongTensor(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[:-3]))

  texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)  # 입력 시퀀스들을 패딩하여 동일한 길이로 맞춤
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from torch import nn
from math import sqrt


class MultiHeadAttention(nn.Module):
  def __init__(self, input_dim, d_model, n_heads):
    super().__init__()

    self.input_dim = input_dim  # 입력 벡터의 차원
    self.d_model = d_model      # 전체 모델 차원 (multi-head attention 출력 차원)
    self.n_heads = n_heads      # 어텐션 헤드의 수

    # 쿼리, 키, 밸류 생성을 위한 선형 변환 레이어들
    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    # 멀티헤드 출력을 다시 하나로 합친 후 사용하는 출력 선형 레이어
    self.dense = nn.Linear(d_model, d_model)

    # 어텐션 스코어에 소프트맥스를 적용하기 위한 함수
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    # 입력 x를 쿼리, 키, 밸류로 선형 변환 (B: 배치, S: 시퀀스 길이, D: 차원)
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    B, S, D = q.shape[0], q.shape[1], self.d_model // self.n_heads # 각 헤드당 차원 계산

    # (B, S, d_model) → (B, n_heads, S, D): 멀티헤드 형태로 변환
    q = q.reshape((B, S, self.n_heads, D)).transpose(1, 2)
    k = k.reshape((B, S, self.n_heads, D)).transpose(1, 2)
    v = v.reshape((B, S, self.n_heads, D)).transpose(1, 2)

    # 어텐션 스코어 계산: Q * K^T → (B, n_heads, S, S)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, H, S, D) * (B, H, D, S) = (B, H, S, S)
    score = score / sqrt(self.d_model) # 어텐션 스코어 정규화 (스케일드 어텐션)

    # 마스크가 있다면, 마스킹된 위치에 매우 작은 값(-1e9)을 더해 softmax 후 영향 없도록 함
    if mask is not None:
      score = score + (mask[:, None] * -1e9)

    # 소프트맥스 적용 → 어텐션 확률로 변환
    score = self.softmax(score)
    # 어텐션 확률을 값 벡터 V에 곱함 → (B, n_heads, S, D)
    result = torch.matmul(score, v)  # (B, H, S, D)

    # (B, n_heads, S, D) → (B, S, d_model): 헤드 차원 합치기
    result = result.transpose(1, 2).reshape((B, S, -1))
    # 최종 출력 선형 레이어 통과
    result = self.dense(result)

    return result

In [ ]:
# Transformer 인코더 레이어 구현
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, n_heads, dff):
    super().__init__()

    self.input_dim = input_dim  # 입력 차원
    self.d_model = d_model      # 모델 차원
    self.n_heads = n_heads      # 어텐션 헤드 수
    self.dff = dff              # FFN의 내부 차원

    # 다중 헤드 어텐션 모듈
    self.sa = MultiHeadAttention(input_dim, d_model, n_heads)
    # Position-wise Feed Forward Network (FFN)
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

    # 첫 번째 LayerNorm (어텐션 후)
    self.norm1 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(0.1)

    # 두 번째 LayerNorm (FFN 후)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout2 = nn.Dropout(0.1)

  def forward(self, x, mask):
    # 어텐션 출력
    x1 = self.sa(x, mask)
    x1 = self.dropout1(x1)
    # 어텐션 결과 + 입력 → Residual + Norm
    x1 = self.norm1(x + x1)

    # FFN 적용
    x2 = self.ffn(x1)
    x2 = self.dropout2(x2)
    # FFN 결과 + Residual + Norm
    x2 = self.norm2(x1 + x2)

    return x

In [ ]:
import numpy as np

# 위치(pos)와 차원(i), 모델 차원(d_model)을 받아 각 위치에 대한 주기적인 각도 값(angle)을 계산
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2]) # 짝수 인덱스(0, 2, 4...)는 sin 적용
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2]) # 홀수 인덱스(1, 3, 5...)는 cos 적용
    pos_encoding = angle_rads[None, ...] # (1, position, d_model) 형태로 차원을 하나 추가하여 배치 차원 포함

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


In [ ]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_heads, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size         # 전체 단어 집합 크기
    self.d_model = d_model               # 임베딩 및 모델 차원
    self.n_heads = n_heads               # 멀티헤드 어텐션의 헤드 수
    self.n_layers = n_layers             # Transformer 인코더 층 수
    self.dff = dff                       # FFN 내부 차원

    self.embedding = nn.Embedding(vocab_size, d_model) # 임베딩 레이어 (토큰 → 벡터)
    # 포지셔널 인코딩 (학습되지 않음, sin/cos 기반)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    # Transformer 인코더 레이어들을 리스트로 저장
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, n_heads, dff) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, len(tokenizer)) # 분류를 위한 출력 레이어

  def forward(self, x):
    seq_len = x.shape[1] # 입력 시퀀스 길이 추출
    mask = (x == tokenizer.pad_token_id)[..., None] # (B, S, 1)

    # 임베딩 적용 후 스케일 조정
    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    # 모든 Transformer 인코더 레이어를 통과
    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0]
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 4, 5, 32)

In [ ]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1)

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [ ]:
n_epochs = 5

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 348.96052265167236
=========> Train acc: 0.000 | Test acc: 0.000
Epoch   1 | Train Loss: 227.64956188201904
=========> Train acc: 0.000 | Test acc: 0.000
Epoch   2 | Train Loss: 105.80481481552124
=========> Train acc: 0.000 | Test acc: 0.000
Epoch   3 | Train Loss: 13.725701160728931
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   4 | Train Loss: 0.7626572623848915
=========> Train acc: 1.000 | Test acc: 1.000
